In [ ]:
import pandas as pd

import tradingPerformance
import plotGraph
import classicalStrategy
from TDQN import TDQN, testing

import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [ ]:
# %run datadownloader.py

In [ ]:
stocks = {
    'Dow Jones' : 'DIA',
    'S&P 500' : 'SPY',
    'NASDAQ 100' : 'QQQ',
    'FTSE 100' : 'EZU',
    'Nikkei 225' : 'EWJ',
    'Google' : 'GOOGL',
    'Apple' : 'AAPL',
    'Meta' : 'META',
    'Amazon' : 'AMZN',
    'Microsoft' : 'MSFT',
    'Nokia' : 'NOK',
    'Philips' : 'PHIA.AS',
    'Siemens' : 'SIE.DE',
    'Baidu' : 'BIDU',
    'Alibaba' : 'BABA',
    'Tencent' : '0700.HK',
    'Sony' : '6758.T',
    'JPMorgan Chase' : 'JPM',
    'HSBC' : 'HSBC',
    'CCB' : '0939.HK',
    'ExxonMobil' : 'XOM',
    'Shell' : 'SHEL',
    'PetroChina' : '0857.HK',
    'Tesla' : 'TSLA',
    'Volkswagen' : 'VOW3.DE',
    'Toyota' : '7203.T',
    'Coca Cola' : 'KO',
    'AB InBev' : 'ABI.BR',
    'Kirin' : '2503.T',
    # 'Twitter' : 'TWTR' # No accessible now
}

## APPLE Stock - TDQN

In [ ]:
stockName, startingDate, splittingDate, endingDate = 'Apple', '2012-01-01', '2018-01-01', '2020-01-01'
stockName = stocks[stockName]

dataTraining_APPL = pd.read_csv('data/'+stockName+'_'+startingDate+'_'+splittingDate+'.csv').set_index('Date')
dataTesting_APPL = pd.read_csv('data/'+stockName+'_'+splittingDate+'_'+endingDate+'.csv').set_index('Date')

In [ ]:
TrainingData_APPL, MainNetwork_APPL = TDQN(stockName, startingDate, splittingDate)

In [ ]:
Training_Data_APPL, _, _ = testing(TrainingData_APPL, dataTraining_APPL, MainNetwork_APPL)
Training_Data_APPL.index = pd.to_datetime(Training_Data_APPL.index)

In [ ]:
plotGraph.render(Training_Data_APPL)
tradingPerformance.displayPerformance(Training_Data_APPL, 'TDQN')

In [ ]:
TestingData_APPL, _, _ = testing(dataTraining_APPL, dataTesting_APPL, MainNetwork_APPL)
TestingData_APPL.index = pd.to_datetime(TestingData_APPL.index)

In [ ]:
plotGraph.render(TestingData_APPL)
tradingPerformance.displayPerformance(TestingData_APPL, 'TDQN')

In [ ]:
plotGraph.plotEntireTrading(Training_Data_APPL, TestingData_APPL)

## TESLA Stock

In [ ]:
stockName, startingDate, splittingDate, endingDate = 'Tesla', '2012-01-01', '2018-01-01', '2020-01-01'
stockName = stocks[stockName]

dataTraining_TSLA = pd.read_csv('data/'+stockName+'_'+startingDate+'_'+splittingDate+'.csv').set_index('Date')
dataTesting_TSLA = pd.read_csv('data/'+stockName+'_'+splittingDate+'_'+endingDate+'.csv').set_index('Date')

In [ ]:
TrainingData_TSLA, MainNetwork_TSLA = TDQN(stockName, startingDate, splittingDate)

In [ ]:
Training_Data_TSLA, _, _ = testing(TrainingData_TSLA, dataTraining_TSLA, MainNetwork_TSLA)
Training_Data_TSLA.index = pd.to_datetime(Training_Data_TSLA.index)
plotGraph.render(Training_Data_TSLA)
tradingPerformance.displayPerformance(Training_Data_TSLA, 'TDQN')

In [ ]:
TestingData_TSLA, _, _ = testing(dataTraining_TSLA, dataTesting_TSLA, MainNetwork_TSLA)
TestingData_TSLA.index = pd.to_datetime(TestingData_TSLA.index)
plotGraph.render(TestingData_TSLA)
tradingPerformance.displayPerformance(TestingData_TSLA, 'TDQN')

In [ ]:
plotGraph.plotEntireTrading(Training_Data_TSLA, TestingData_TSLA)

# Simulation sur toutes les stratégies (sur 5 actions)

In [ ]:
stock_tested = ['Apple', 'Tesla', 'S&P 500', 'Tencent', 'Volkswagen']
strategies = ['TDQN', 'Buy and Hold', 'Sell and Hold', 'MA_TF', 'MA_MR']
score_stocks = {stock: 0 for stock in stock_tested}

In [ ]:
startingDate = '2012-01-01'
splittingDate = '2018-01-01'
endingDate = '2020-01-01'

for stock in stock_tested:
    scores = {strategy:0 for strategy in strategies}
    dataTraining = pd.read_csv('data/'+stocks[stock]+'_'+startingDate+'_'+splittingDate+'.csv').set_index('Date')
    dataTesting = pd.read_csv('data/'+stocks[stock]+'_'+splittingDate+'_'+endingDate+'.csv').set_index('Date')
    TrainingData, MainNetwork = TDQN(stocks[stock], startingDate, splittingDate)
    TestedData, _, _ = testing(dataTraining, dataTesting, MainNetwork)
    score_TDQN = tradingPerformance.computeSharpeRatio(TestedData)
    scores['TDQN'] = score_TDQN
    scores['Buy and Hold'] = tradingPerformance.computeSharpeRatio(classicalStrategy.buy_and_hold(dataTesting))
    scores['Sell and Hold'] = tradingPerformance.computeSharpeRatio(classicalStrategy.sell_and_hold(dataTesting))
    scores['MA_TF'] = tradingPerformance.computeSharpeRatio(classicalStrategy.MA_TF(dataTraining, dataTesting))
    scores['MA_MR'] = tradingPerformance.computeSharpeRatio(classicalStrategy.MA_MR(dataTraining, dataTesting))
    print('Stock:', stock)
    print('Scores:', scores)
    TestedData.index = pd.to_datetime(TestedData.index)
    plotGraph.render(TestedData, stock)
    score_stocks[stock] = scores

In [ ]:
score_stocks